In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import torch
torch.cuda.is_available()

True

Скачиваем mergekit
Инструкция к скачиванию и документацию можно найти по ссылке:
https://github.com/cg123/mergekit

In [ ]:
!git clone https://github.com/cg123/mergekit.git

Cloning into 'mergekit'...
remote: Enumerating objects: 1100, done.
remote: Counting objects: 100% (609/609), done.
remote: Compressing objects: 100% (240/240), done.
remote: Total 1100 (delta 473), reused 422 (delta 364), pack-reused 491
Receiving objects: 100% (1100/1100), 303.24 KiB | 10.46 MiB/s, done.
Resolving deltas: 100% (741/741), done.


In [ ]:
%cd /content/mergekit
%pwd
#!python3 -m pip install --upgrade pip
!pip install -e . # install the package and make scripts available


/content/mergekit
Obtaining file:///content/mergekit
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.1 MB/s eta 0:00:00
  Building editable for mergekit (pyproject.toml) ... done
  Created wheel for mergekit: filename=mergekit-0.0.4.1-0.editable-py3-none-any.whl size=9646 sha256=f127f07ee66fd84685e1e86f1504c3c4709193555d6f1ba81

In [ ]:
!pip list

In [ ]:
!mergekit-yaml --help

Usage: mergekit-yaml [OPTIONS] CONFIG_FILE OUT_PATH

Options:
  -v, --verbose                   Verbose logging
  --allow-crimes / --no-allow-crimes
                                  Allow mixing architectures  [default: no-
                                  allow-crimes]
  --transformers-cache TEXT       Override storage path for downloaded models
  --lora-merge-cache TEXT         Path to store merged LORA models
  --cuda / --no-cuda              Perform matrix arithmetic on GPU  [default:
                                  no-cuda]
  --low-cpu-memory / --no-low-cpu-memory
                                  Store results and intermediate values on
                                  GPU. Useful if VRAM > RAM  [default: no-low-
                                  cpu-memory]
  --out-shard-size SIZE           Number of parameters per output shard
                                  [default: 5B]
  --copy-tokenizer / --no-copy-tokenizer
                                  Copy a tokenizer to the o

In [ ]:
OUTPUT_PATH = "./merged"  # folder to store the result in
LORA_MERGE_CACHE = "/tmp"  # change if you want to keep these for some reason. /tmp it is temporary directory for temporary files, which create when we merge the models
CONFIG_YML = "/content/mergekit/examples/ties.yml"  # merge configuration file
COPY_TOKENIZER = True  # you want a tokenizer? yeah, that's what i thought
LAZY_UNPICKLE = False  # experimental low-memory model loader
LOW_CPU_MEMORY = False  # enable if you somehow have more VRAM than RAM+swap

In /example you can see merge_methods like Ties, Linear, SLERP and passthrough, but we can use DARE for merge two LLM's

In [ ]:
import torch # import the torch package
import yaml # import the yaml package, because we will need work with config.yml file

from mergekit.merge import MergeOptions, run_merge # import mergekit.merge.(MergeOptions, run_merge)
from mergekit.config import MergeConfiguration # import mergekit.config.MergeConfiguration

with open(CONFIG_YML, "r", encoding='utf-8') as fp: # open the CONFIG_YML and encode that file in utf-8
  merge_config = MergeConfiguration.model_validate(yaml.safe_load(fp)) # merge_config

run_merge(
    merge_config,
    out_path=OUTPUT_PATH,
    options = MergeOptions(
        lora_merge_cache=LORA_MERGE_CACHE,
        cuda=torch.cuda.is_available(),
        copy_tokenizer=COPY_TOKENIZER,
        lazy_unpickle=LAZY_UNPICKLE,
        low_cpu_memory=LOW_CPU_MEMORY,
    )
)
print ("done")
